In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import gensim
from gensim.utils import simple_preprocess
from gensim import corpora, models
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import re
import os
np.random.seed(2018)
import nltk
# nltk.download('wordnet')
from collections import Counter

import sys
sys.path.insert(1,"../../midterm_election_semantics_2022/")
from helpers import clean_tweets

In [2]:
tweets = pd.read_excel("./data/authored_politician_tweets_11_14.xlsx")

In [3]:
tweets.head()

,id,tweet_url,created_at,parsed_created_at,user_screen_name,text,tweet_type,coordinates,hashtags,media,...,user_favourites_count,user_followers_count,user_friends_count,user_listed_count,user_location,user_name,user_statuses_count,user_time_zone,user_urls,user_verified
0,1591119455538806784,https://twitter.com/MarkAmodeiNV2/status/15911...,Fri Nov 11 17:23:34 +0000 2022,2022-11-11T17:23:34+00:00,MarkAmodeiNV2,LVRJ reporter Gary Martin was a thoughtful jou...,original,NaN,NaN,NaN,...,160,16465,1095,818,"Carson City, NV",RepMarkAmodei,1491,NaN,http://amodei.house.gov,True
1,1592202455424851968,https://twitter.com/mercedes4nevada/status/159...,Mon Nov 14 17:07:01 +0000 2022,2022-11-14T17:07:01+00:00,mercedes4nevada,Our campaign proved this district is full of p...,original,NaN,NaN,NaN,...,602,1405,142,23,"Sparks, NV",Elizabeth Mercedes Krause,406,NaN,http://mercedes4nevada.com,False
2,1591629899181477889,https://twitter.com/mercedes4nevada/status/159...,Sun Nov 13 03:11:53 +0000 2022,2022-11-13T03:11:53+00:00,mercedes4nevada,@PhotoCassidy @CortezMasto Absolutely! That’s ...,reply,NaN,NaN,NaN,...,602,1405,142,23,"Sparks, NV",Elizabeth Mercedes Krause,406,NaN,http://mercedes4nevada.com,False
3,1591996316716089344,https://twitter.com/RepRosendale/status/159199...,Mon Nov 14 03:27:54 +0000 2022,2022-11-14T03:27:54+00:00,RepRosendale,How can we hold leadership elections when 20 r...,quote,NaN,NaN,NaN,...,231,15766,269,320,"Montana, USA",Matt Rosendale,1823,NaN,https://rosendale.house.gov/,True
4,1591103383842455552,https://twitter.com/RepRosendale/status/159110...,Fri Nov 11 16:19:42 +0000 2022,2022-11-11T16:19:42+00:00,RepRosendale,"On Veterans Day, we honor the men and women wh...",original,NaN,NaN,https://pbs.twimg.com/media/FhS78XJWYAMXGWD.jpg,...,231,15766,269,320,"Montana, USA",Matt Rosendale,1823,NaN,https://rosendale.house.gov/,True


In [4]:
tweets = clean_tweets(tweets)

../../midterm_election_semantics_2022\helpers.py:11: FutureWarning: The default value of regex will change from True to False in a future version.
  data['clean_text'] = data['clean_text'].str.replace('\d+', '')


In [5]:
stemmer = SnowballStemmer("english")
def lemmatize_stemming(text):
        return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [6]:
tweets['mentions'] = tweets['clean_text'].apply(lambda x: re.findall(r"@[a-zA-Z0-9_]*",x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x: re.sub(r"@[a-zA-Z0-9_]*","", x))
tweets['clean_text'] = tweets['clean_text'].apply(lambda x: re.sub(r"[^A-Za-z]+"," ", x))
tweets = tweets.loc[tweets['clean_text'] != ""]
processed_tweets = tweets['clean_text'].map(preprocess)

In [7]:
if "models" in os.listdir("../"):
    dictionary = gensim.corpora.dictionary.Dictionary.load("../models/topic_model_gensim.id2word")
    lda_model_tfidf = gensim.models.LdaMulticore(id2word=dictionary).load("../models/topic_model_gensim")
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]
else:
    dictionary = gensim.corpora.Dictionary(processed_tweets)
    dictionary.filter_extremes(no_below=400, no_above=0.5, keep_n=50000)
    bow_corpus = [dictionary.doc2bow(doc) for doc in processed_tweets]
    tfidf = models.TfidfModel(bow_corpus)
    corpus_tfidf = tfidf[bow_corpus]


    lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=60, id2word=dictionary, passes=2, workers=4)
    # for idx, topic in lda_model_tfidf.print_topics(-1):
    #     print('Topic: {} Word: {}'.format(idx, topic))

In [17]:
# # Compute Perplexity
# print('\nPerplexity: ', lda_model_tfidf.log_perplexity(corpus_tfidf[:100000]))  
# a measure of how good the model is. lower the better.


Perplexity:  -10.126415091013172


In [18]:
# for n in [25,30,35,40,45,50,55,60]:
#     print(n)
#     lda_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics=n, id2word=dictionary, passes=2, workers=4)
#     print('Perplexity: ', lda_model.log_perplexity(corpus_tfidf[:100000]))
    

# Vizualizing the topics

In [11]:
import pyLDAvis.gensim_models

lda_display = pyLDAvis.gensim_models.prepare(lda_model_tfidf, corpus_tfidf, dictionary, sort_topics=False)
pyLDAvis.display(lda_display)

c:\Users\thatb\anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [20]:
# lda_model_tfidf.save('../models/topic_model_gensim')

c:\Users\thatb\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


# Applying to tweets



In [8]:
topic_list = []
prob_list = []
for doc in corpus_tfidf:
    topics = lda_model_tfidf.get_document_topics(doc)
    probs = [topic[1] for topic in topics]
    topic_ids = [topic[0] for topic in topics]
    max_index = probs.index(max(probs))

    topic_list.append(topic_ids[max_index])
    prob_list.append(probs[max_index])

In [9]:
tweets['topic'] = topic_list
tweets['topic_prob'] = prob_list

In [10]:
tweets.to_csv("./data/clean_topics_tweets_final.csv")

In [14]:
Counter(list(tweets['topic'])).most_common(20)

c:\Users\thatb\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0, 8935),
 (53, 6446),
 (9, 6214),
 (45, 5966),
 (59, 5786),
 (6, 5577),
 (30, 5427),
 (23, 5171),
 (39, 5066),
 (38, 4499),
 (25, 4340),
 (47, 4214),
 (11, 4113),
 (37, 4014),
 (24, 3825),
 (49, 3802),
 (56, 3767),
 (51, 3638),
 (22, 3521),
 (27, 3493)]

In [57]:
test_t = tweets.loc[tweets['topic'] == 27][['text','topic_prob']].sort_values('topic_prob', ascending=False)[:10]
[print("\n", test) for test in test_t['text']]


 We have an amazing team that I’m so proud of! While I was in #Philly, they were in Aston &amp; then Newtown Square #Delco — working hard, listening, &amp; growing our movement. They believe in our message. They’re building a better #PA5 with friends &amp; neighbors. We hope you’ll join us. https://t.co/QClewlZ1Rn

 Our first mailer has hit mailboxes! Spread the word to your friends, neighbors, and family! We don’t care where you live or what party you are — we want you on the team! Check out our website to donate and get involved today! 👇🏻👇🏻👇🏻 #galluchforcongress  https://t.co/ciq1Y8DKxQ https://t.co/99vhjO1MKL

 Getting our whole community involved in working towards a better future is what a great democracy like ours is built on. 1 person = 1 vote. Make your voting plan and ask others what their plan is. Not sure what to do? DM our team and volunteer! https://t.co/dqvMTjE8rQ

 We don't know about you, but we think it would be amazing to have a Democrat in the U.S. House of Represen

c:\Users\thatb\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[None, None, None, None, None, None, None, None, None, None]

In [58]:
[print(dictionary[topic[0]]) for topic in lda_model_tfidf.get_topic_terms(27)]

weekend
appreci
champion
pack
detail
congressman
view
involv
listen
websit


c:\Users\thatb\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[None, None, None, None, None, None, None, None, None, None]

In [59]:
topic_name_map = {0 :"military, russia, ukraine, putin",
					53:"economy, job, infrastructure",
					9 :"pro-abortion",
					45:"American rescue Plan, small business",
					59:"Armed forces, veterans",
					6 :"education, children",
					30:"lower health care, IRA",
					23:"supreme court, Roe v Wade, LGBTQ",
					39:"anti-biden, layoffs, border crisis",
					38:"assorted thanks",
					25:"police",
					47:"disability, child-care",
					11:"?",
					37:"unions, labor",
					24:"immigration, captiol riots, climate change",
					49:"deomcrats, win, nevada, north carolina",
					56:"endorsement",
					51:"general celebration",
					22:"Trump, Biden",
					27:"?"}

c:\Users\thatb\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
